## Notes

In order to make the code work, this notebook should be inside the same folder as the images.
A sub-folder named "Foreground" should also exist in the same folder as the notebook and the images.

## Imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import itemfreq

## Background substraction

My first idea was to extract the foreground, which is the moto. To do so, I designed the fg_extraction function, that uses the createBackgroundSubstractorMOG2 function of OpenCV. The results were noisy, so I created the post_processing function, subsequently applying a median filer, a threshold, and the erode and dilate function, to have the foreground mask with less noise.

In [2]:
def post_processing(img):
    median = cv2.medianBlur(img,5)
    thresh = cv2.threshold(median, 200, 255, cv2.THRESH_BINARY)[1]   
    thresh = cv2.erode(thresh, None, iterations=2)
    post = cv2.dilate(thresh, None, iterations=4)
    return post

In [3]:
def fg_extraction():
    """
    This function extracts the foreground from the background and delivers black and white images, the foreground being in white.
    """
    
    fgbg = cv2.createBackgroundSubtractorMOG2()
    for k in range(2):
        for i in range(30,192):
            if i<100:
                img_name="00"+str(i)
            else:
                img_name="0"+str(i)
            img = cv2.imread(img_name+'.jpg')
            fgmask = fgbg.apply(img)
            post=post_processing(fgmask)
            cv2.imwrite('Foreground\\fg_'+img_name+'.jpg',fgmask)
            cv2.imwrite('Foreground\\fg_post_'+img_name+'.jpg',post)

Once the foreground extracted, I need to count the big blobs inside the mask. To do so, we look for the connected components of the mask. If the size of the blob is big enough, we count the blob.

In [4]:
def blob_count(img,threshold=50,connectivity=4):
    img = cv2.imread(img,0) #0 is needed to open the image in grayscale.
    img[0:300,1400:]=0 #remove top_right corner
    output = cv2.connectedComponentsWithStats(img, connectivity, cv2.CV_32S)
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    centroids = output[3]
    count=0
    blobs=[]
    for label in range(1,num_labels):
        width = stats[label, cv2.CC_STAT_WIDTH]
        height = stats[label, cv2.CC_STAT_HEIGHT]
        x = stats[label, cv2.CC_STAT_LEFT]
        y = stats[label, cv2.CC_STAT_TOP]
        if width>=threshold and height>=threshold: #can instead use the number of white pixels.
            #roi = img[y:int(y+height), x:int(x+width)]
            #cv2.imwrite('blob_'+str(label)+'.jpg', roi)
            count+=1
            blobs.append([x, y, width, height])
    return count, blobs

The following function delivers the output where are printed how many motos are on each picture.

In [5]:
def is_there_motos():
    for i in range(30,192):
        if i<100:
            img_name="00"+str(i)
        else:
            img_name="0"+str(i)
        count, blobs = blob_count('Foreground\\fg_post_'+img_name+'.jpg')
        if count==0:
            print('There isn\'t any moto in the picture {}.'.format(img_name))
        elif count==1:
            print('There is one moto in the picture {}.'.format(img_name))
        else:
            print('There are {} motos in the picture {}.'.format(count, img_name))
        if count>0:
            img = cv2.imread(img_name+'.jpg')
            fg= cv2.imread('Foreground\\fg_post_'+img_name+'.jpg',0)
            fg_color=cv2.bitwise_and(img,img,mask = fg)
            for blob in blobs:
                x,y,width,height=blob
                img_blob=fg_color[y:int(y+height), x:int(x+width)]
                #most_present_color=find_color(img)
    

## Execution

In [7]:
fg_extraction()

In [8]:
is_there_motos()

There isn't any moto in the picture 0030.
There isn't any moto in the picture 0031.
There isn't any moto in the picture 0032.
There isn't any moto in the picture 0033.
There isn't any moto in the picture 0034.
There isn't any moto in the picture 0035.
There isn't any moto in the picture 0036.
There is one moto in the picture 0037.
There is one moto in the picture 0038.
There is one moto in the picture 0039.
There is one moto in the picture 0040.
There is one moto in the picture 0041.
There is one moto in the picture 0042.
There is one moto in the picture 0043.
There is one moto in the picture 0044.
There is one moto in the picture 0045.
There is one moto in the picture 0046.
There is one moto in the picture 0047.
There is one moto in the picture 0048.
There is one moto in the picture 0049.
There is one moto in the picture 0050.
There is one moto in the picture 0051.
There is one moto in the picture 0052.
There is one moto in the picture 0053.
There is one moto in the picture 0054.
Ther

## Test area

In [38]:
img_name='0114'
img = cv2.imread(img_name+'.jpg')
print(len(img))
print(len(img[0]))
tr_corner=img[0:300,1400:]
img[0:300,1400:]=0
cv2.imwrite('test.jpg',img)
    
    

1080
1920


True

In [8]:
img_name='0114'
img = cv2.imread(img_name+'.jpg')
fg= cv2.imread('Foreground\\fg_post_'+img_name+'.jpg',0)
cv2.imwrite('test.jpg', cv2.bitwise_and(img,img,mask = fg))

True